# [GoNB](https://github.com/janpfeifer/gonb) WASM (WebAssembly) Examples 

### Flag Parsing in Wasm

**GoNB** exports arguments passed in a slice for Wasm programs in the global variable `GonbWasmArgs`.

This allows for parsing into flags where desired:

In [2]:
!*rm -f go.work && go work init
!*go work use . "${HOME}/Projects/gonb"
%goworkfix

	- Added replace rule for module "github.com/janpfeifer/gonb" to local directory "/home/janpf/Projects/gonb".


In [16]:
import (
        "github.com/janpfeifer/gonb/gonbui"
)
%%
gonbui.ConnectToJupyterServer()

In [17]:
!set | grep GONB

BASH_EXECUTION_STRING='set | grep GONB'
GONB_DIR=/home/janpf/Projects/gonb/examples
GONB_JUPYTER_ROOT=/home/janpf
GONB_JUPYTER_SESSION_ID=99cf1321-773b-4f75-8efb-67a7b804a688
GONB_PIPE=/tmp/gonb_pipe_743942013
GONB_TMP_DIR=/tmp/gonb_ce4b7d59


In [2]:
%wasm
import (
    "flag"
    "fmt"
    "os"
    
    "github.com/janpfeifer/gonb/gonbui/wasm"
)

var flagName = flag.String("name", "", "enter your name")

%% --name=World
fmt.Printf("os.Args=%v\n", os.Args)
flag.Parse()
fmt.Printf("--name=%s\n", *flagName)
// wasm.ParseFlags(GonbWasmArgs)  // Parse flags form the args passed in GonbWasmArgs.
wasm.Alertf("Hello %s!", *flagName)

### Simplest Widget: Sum Two Numbers

An example of making interactive widgets using Go and WASM inside GoNB.

First we define a function that sum numbers given in strings. 
Notice we run it normally, not yet in Wasm.

In [8]:
func SumStr(aStr, bStr string) string {
    a, err := strconv.Atoi(aStr)
    if err != nil {
        return "#invalid"
    }
    b, err := strconv.Atoi(bStr)
    if err != nil {
        return "#invalid"
    }
    x := a+b
    if x != 42 {
        return strconv.Itoa(x)
    } else {
        return "life, the universe and everything ..."
    }
}

%%
fmt.Println("1 + 2 =", SumStr("1", "2"))

1 + 2 = 3


Now, we define the Wasm program that creates the input fields and listen to its changes, updating the sum:

In [10]:
%wasm
import (
    "github.com/janpfeifer/gonb/gonbui/wasm"
    "syscall/js"
)

%%
// mainDiv := document.Call("getElementById", GONB_WASM_DIV_ID)
// mainDiv.Set("innerHTML", "<b>Hello World!</b>")
div := wasm.ById(GonbWasmDivId)
inputA := wasm.AsInput(wasm.NewElem("input", "width=10"))
inputB := wasm.AsInput(wasm.NewElem("input", "width=10"))
result := wasm.AsInput(wasm.NewElem("input", "width=10", "readonly"))

wasm.Append(div, inputA)
wasm.AppendHTML(div, " + ")
wasm.Append(div, inputB)
wasm.AppendHTML(div, " + ")
wasm.Append(div, result)

// Sum numbers whenever they change.
updater := func(_ wasm.EventCompatible) {
    result.SetValue(SumStr(inputA.Value(), inputB.Value()))
}
wasm.On(inputA, "keyup", updater)
wasm.On(inputB, "keyup", updater)

// Never return, otherwise the Go wasm program stops working and won't update the result.
wasm.WaitForever()